
# Column Transformer with Mixed Types


This example illustrates how to apply different preprocessing and feature
extraction pipelines to different subsets of features, using
`ColumnTransformer`. This is particularly handy for the
case of datasets that contain heterogeneous data types, since we may want to
scale the numeric features and one-hot encode the categorical ones.

In this example, the numeric data is standard-scaled after mean-imputation,
while the categorical data is one-hot encoded after imputing missing values
with a new category (``'missing'``).

In addition, we show two different ways to dispatch the columns to the
particular pre-processor: by column names and by column data types.

Finally, the preprocessing pipeline is integrated in a full prediction pipeline
using `Pipeline`, together with a simple classification
model.


In [326]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

#### Step 1: Read the data set `Titanic.csv`
Print the column names. Drop 'boat','body','cabin','home.dest','name','ticket','sibsp','parch'

In [327]:
df = pd.read_csv('Titanic.csv')

In [328]:
df.shape

(1309, 14)

In [329]:
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [330]:
df.head(7)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48,0,0,19952,26.55,E12,S,3,?,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63,1,0,13502,77.9583,D7,S,10,?,"Hudson, NY"


In [331]:
df2 = df.drop(['boat','body','cabin','home.dest','name','ticket','sibsp','parch'], axis=1)

In [332]:
df2.columns

Index(['pclass', 'survived', 'sex', 'age', 'fare', 'embarked'], dtype='object')

#### Step 2: Calculate duplicates
- Report the size of the dataset
- Report if there are any duplicates using `duplicated()` method of pandas dataframe
- List all duplicate rows
- Drop all duplicate rows using `dop_duplicates` method of pandas dataframe
- Report the new shape of the dataset

In [333]:
df2.shape

(1309, 6)

In [334]:
df2.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1304    False
1305     True
1306    False
1307    False
1308    False
Length: 1309, dtype: bool

In [335]:
df2.drop_duplicates(inplace=True)

In [336]:
df2.shape

(1094, 6)

#### Step 3: Explore the dataset for missing values
- Assign `survived' as target variable `y` and the rest as `X`
- Print info about `X` to see if there are any null values and the type of the features
- Check if all columns have only alpha numeric characters. To do that you need to use `isalnum()` method of Python which works on strings. Before using this method. we need to change all features into string using `astype('str')`
- Replace all `?` characters with `np.nan`
- Now, turn 'age','fare' into numeric again using `apply(pd.to_numeric) `
- Print the first five samples of `X`
- Summarize the number of unique values in each column using `nunique()`

In [337]:
X = df2.drop(columns=['survived'], axis=1)
y = df2['survived']

In [338]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [339]:
cols = X.columns
X = X[cols].astype(str)

In [340]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   object
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: object(5)
memory usage: 51.3+ KB


In [341]:
X.isnull().sum()

pclass      0
sex         0
age         0
fare        0
embarked    0
dtype: int64

In [342]:
X.head()

,pclass,sex,age,fare,embarked
0,1,female,29,211.3375,S
1,1,male,0.9167,151.55,S
2,1,female,2,151.55,S
3,1,male,30,151.55,S
4,1,female,25,151.55,S


In [343]:
# go to row 3 (order from 0 to n) and column 'age' to find element
X.loc[3,'age']

'30'

In [344]:
# go to row 3 (index from 0 to n) and column 2 to find element
X.iloc[3,2]

'30'

In [345]:
# get all element/rows at column 2
X.iloc[:,2].str.isalnum() 

0        True
1       False
2        True
3        True
4        True
        ...  
1301    False
1304    False
1306    False
1307     True
1308     True
Name: age, Length: 1094, dtype: bool

In [346]:
# get all elements/rows at column 2 and sum it
(~X.iloc[:,2].str.isalnum()).sum()

172

In [347]:
X[X.iloc[:,2].str.isalnum() == True]

,pclass,sex,age,fare,embarked
0,1,female,29,211.3375,S
2,1,female,2,151.55,S
3,1,male,30,151.55,S
4,1,female,25,151.55,S
5,1,male,48,26.55,S
...,...,...,...,...,...
1298,3,male,36,9.5,S
1299,3,male,27,14.4542,C
1300,3,female,15,14.4542,C
1307,3,male,27,7.225,C


In [348]:
X.replace({'?':np.nan})

,pclass,sex,age,fare,embarked
0,1,female,29,211.3375,S
1,1,male,0.9167,151.55,S
2,1,female,2,151.55,S
3,1,male,30,151.55,S
4,1,female,25,151.55,S
...,...,...,...,...,...
1301,3,male,45.5,7.225,C
1304,3,female,14.5,14.4542,C
1306,3,male,26.5,7.225,C
1307,3,male,27,7.225,C


In [349]:
type(np.nan)

float

In [350]:
X.iloc[:,2].str.isalnum()[:20]

0      True
1     False
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16     True
17     True
18     True
19     True
Name: age, dtype: bool

In [351]:
type(np.nan)

float

In [352]:
X['age'] = X['age'].replace({'?':np.nan})
X['fare'] = X['fare'].replace({'?':np.nan})
X[['age','fare']]=X[['age','fare']].apply(pd.to_numeric)

In [365]:
ty1 = {'age':float, 'fare':float}
X = X.astype(ty1)

In [366]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1094 non-null   object 
 1   sex       1094 non-null   object 
 2   age       966 non-null    float64
 3   fare      1093 non-null   float64
 4   embarked  1094 non-null   object 
dtypes: float64(2), object(3)
memory usage: 83.6+ KB


In [367]:
X['embarked'] = X['embarked'].astype(str)

In [368]:
np.unique(X['embarked'])

array(['?', 'C', 'Q', 'S'], dtype=object)

#### Step 4: Use `ColumnTransformer` by selecting column by names

##############################################################################
 
 We will train our classifier with the following features:

 Numeric Features:

 * ``age``: float;
 * ``fare``: float.

 Categorical Features:

 * ``embarked``: categories encoded as strings ``{'C', 'S', 'Q'}``;
 * ``sex``: categories encoded as strings ``{'female', 'male'}``;
 * ``pclass``: ordinal integers ``{1, 2, 3}``.

 Create the preprocessing pipelines for both numeric and categorical data.
 - For numeric features, create a pipeline with using imputer with median strategy and standard scaler
 - For categorical features, use imputer with most frequent and onehot encoder
 
 Use columntransformer to transform the features
 
 Append the preprocessor to the KNeighborsClassifier


In [369]:
numeric_features = ['age', 'fare']
categorical_features = ['embarked', 'sex', 'pclass']

In [377]:
numeric_transformer = Pipeline(steps = [
    ('numimp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps = [
    ('catimp', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])

In [383]:
tr = ColumnTransformer(transformers = [
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
    ])

tr

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('numimp',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare']),
                                ('cat',
                                 Pipeline(steps=[('catimp',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'sex', 'pclass'])])

In [ ]:
clf = Pipeline(steps=[ ('prep', tr), ('knn', KNeighborsClassifier()) ])

clf

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('numimp',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('catimp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

In [400]:
# This was the error in class, the reason it didn't work was because:
# I wrote X_train, y_train, ... in the wrong order
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

In [401]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, 1003 to 1120
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    820 non-null    object 
 1   sex       820 non-null    object 
 2   age       722 non-null    float64
 3   fare      820 non-null    float64
 4   embarked  820 non-null    object 
dtypes: float64(2), object(3)
memory usage: 38.4+ KB


In [402]:
clf.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('numimp',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('catimp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

In [403]:
clf.score(X_test, y_test)

0.7737226277372263

Using the prediction pipeline in a grid search

##############################################################################

 Grid search can also be performed on the different preprocessing steps
 defined in the `ColumnTransformer` object, together with the classifier's
 hyperparameters as part of the `Pipeline`.
 We will search for both the imputer strategy of the numeric preprocessing
 and the number of neighbor parameter of the kneighbors classifier using
 `GridSearchCV`.



In [404]:
params = { 'prep__num__numimp__strategy':['mean', 'median'], 'knn__n_neighbors':np.arange(1,30,2) }

In [405]:
mygrid = GridSearchCV(clf, param_grid = params, cv=5)

In [406]:
mygrid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('numimp',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('catimp',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['embarked',
                                                                          'sex',
                                                                          'pclass'])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]),
                         'prep__num__numimp__strategy': ['mean', 'median']})

In [407]:
# should return mean as best strategy
mygrid.best_params_

{'knn__n_neighbors': 9, 'prep__num__numimp__strategy': 'mean'}